# Continue Training Sentiment Analysis Model

Notebook này cho phép:
1. Khôi phục model từ checkpoint
2. Tiếp tục train với dữ liệu mới
3. Đánh giá và lưu kết quả training

In [7]:
import os
import sys
project_root = os.path.dirname(os.path.dirname(os.path.abspath('')))
sys.path.append(project_root)

from src.config import Config
from src.models.model_trainer import EnhancedModelTrainer
from src.data.data_loader import DataLoader
from src.data.preprocessor import DataPreprocessor
from src.features.feature_engineering import FeatureExtractor
import pandas as pd
import numpy as np
from IPython.display import display, Markdown

In [8]:
# Khởi tạo config và components
config = Config()
language = 'vi'  # hoặc 'en'

trainer = EnhancedModelTrainer(language, config)
preprocessor = DataPreprocessor(language, config)
feature_extractor = FeatureExtractor(language, config)

Loaded feature extractor with None dimensions


In [9]:
# Liệt kê các checkpoints có sẵn
checkpoints = trainer.list_checkpoints()

display(Markdown("## Available Checkpoints"))
for i, cp in enumerate(checkpoints):
    print(f"{i+1}. {cp['filename']}")
    print(f"   Timestamp: {cp['timestamp']}")
    print(f"   Epoch: {cp['epoch']}")
    print(f"   Score: {cp['metrics']:.4f if cp['metrics'] else 'N/A'}\n")

## Available Checkpoints

1. vi_checkpoint.pkl
   Timestamp: Unknown
   Epoch: Unknown


TypeError: unsupported format string passed to NoneType.__format__

In [ ]:
# Chọn checkpoint để tiếp tục train
checkpoint_idx = int(input("Enter checkpoint number (or press Enter for latest): ") or 0) - 1
num_epochs = int(input("Enter number of epochs to train: "))

if 0 <= checkpoint_idx < len(checkpoints):
    checkpoint_name = checkpoints[checkpoint_idx]['filename']
else:
    checkpoint_name = None

ValueError: invalid literal for int() with base 10: ''

In [ ]:
# Tải dữ liệu training
train_file = input("Enter training data file path: ")
df = pd.read_csv(train_file)

# Tiền xử lý dữ liệu
processed_df = preprocessor.preprocess(df)
print(f"\nProcessed {len(processed_df)} samples")

# Chuẩn bị features và labels
X_train = processed_df["cleaned_text"] 
y_train = processed_df["label"]

In [ ]:
# Tiếp tục training
model, metrics = trainer.continue_training(
    X_train=X_train,
    y_train=y_train, 
    checkpoint_name=checkpoint_name,
    num_epochs=num_epochs
)

In [ ]:
# Hiển thị kết quả training
import matplotlib.pyplot as plt

for model_name, model_metrics in metrics['models'].items():
    plt.figure(figsize=(10,5))
    
    train_scores = model_metrics['train_scores']
    valid_scores = model_metrics['valid_scores']
    epochs = range(1, len(train_scores) + 1)
    
    plt.plot(epochs, train_scores, 'o-', label='Training')
    plt.plot(epochs, valid_scores, 's--', label='Validation')
    
    plt.title(f'{model_name} Learning Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    print(f"\n{model_name} Final Metrics:")
    print(f"Best Score: {model_metrics['best_score']:.4f}")
    print(f"Training Time: {model_metrics['training_time']:.2f}s")

## Hướng dẫn sử dụng

1. Chọn ngôn ngữ ('vi' hoặc 'en') trong cell thứ 2
2. Chọn checkpoint muốn tiếp tục train từ danh sách
3. Nhập số epoch muốn train thêm 
4. Nhập đường dẫn file dữ liệu training (.csv)
5. Theo dõi quá trình training và kết quả

Model và metrics sẽ được tự động lưu theo checkpoint.